In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_excel("../data/Dailylog data -lifelog/Lifestyle - Sleep.xlsx")

In [3]:
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d %H:%M:%S')
df['Bedtime'] = pd.to_datetime(df['Bedtime'], format = '%Y-%m-%d %H:%M:%S')
df['Wake up time'] = pd.to_datetime(df['Wake up time'], format = '%Y-%m-%d %H:%M:%S')

In [4]:
df.rename(columns={"Non-identifying keys":"id","Measurement (-1: no value, 0: unknown sleep, 1: wake, 2: light sleep, 3: light sleep, 4: deep sleep)":"Measurement"},inplace=True)

In [5]:
df.isna().sum()

id                   0
Date                 0
Measurement types    0
Units of measure     0
Measurement          0
Bedtime              0
Wake up time         0
dtype: int64

In [6]:
df["Measurement"] = df["Measurement"].apply(lambda x: list(map(int, re.findall(r'\d', x))))

In [7]:
df_exploded=df.explode("Measurement")

In [8]:
sleep_patterns=pd.pivot_table(
    df_exploded,
    index=["id","Date"],
    columns="Measurement",
    aggfunc="size",
    fill_value=0
)
cols = {-1: "no value", 0: "unknown sleep", 1: "wake", 2: "light sleep_1", 3: "light sleep_2", 4: "deep sleep"}
sleep_patterns.rename(columns=cols,inplace=True)

In [9]:
sleep_patterns.drop(columns=9,inplace=True)

In [10]:
df=pd.merge(df,sleep_patterns,on=["id","Date"])

In [11]:
df.columns=df.columns.str.lower()

In [12]:
df["sleep_duration"]=df["wake up time"]-df["bedtime"]

In [13]:
df["total_sleep_time"]=df["unknown sleep"]+df["light sleep_1"]+df["light sleep_2"]+df["deep sleep"]

In [14]:
df["sleep_efficiency"]=(df["total_sleep_time"]/df["sleep_duration"].dt.total_seconds())*100

In [15]:
df["sleep_duration"] = df["sleep_duration"].dt.total_seconds()/3600

In [16]:
df["sleep_in"]=df["date"]-df["bedtime"]
df["sleep_out"]=df["date"]-df["wake up time"]

df["sleep_in"]=df["sleep_in"].dt.total_seconds()/3600
df["sleep_out"]=df["sleep_out"].dt.total_seconds()/3600

In [17]:
df.drop(columns=["measurement","units of measure","measurement types","total_sleep_time","bedtime","wake up time"],inplace=True)

In [18]:
print(df[["id","date"]].duplicated().sum())
df = df.drop_duplicates(subset=["id", "date"], keep='first')
print(df[["id","date"]].duplicated().sum())

22
0


In [19]:
df.set_index(["id","date"],inplace=True)

In [20]:
df.columns=df.columns.str.lower()
df.to_csv("../data_processed/daily_lifelog/sleep.csv")